# Evaluation of the Model

In [4]:
import os
import sys

parent = os.path.dirname(os.path.abspath(os.getcwd()))
if parent not in sys.path:
    sys.path.append(parent)
    
parent

'c:\\Users\\aless\\Documents\\git\\XAI-Cancer-Diagnosis\\XAI-Cancer-Diagnosis'

# Data and Models

In [5]:
train_ind, val_ind = [], []
src_folder = os.path.join(parent, "data", "processed",
                            "BACH_TRAIN")
graph_split = os.path.join(src_folder, "graph_ind.txt")
with open(graph_split, "r") as f:
    l1 = f.readline().strip()
    l2 = f.readline().strip()
    train_ind = list(map(int, l1[1:-1].split(",")))
    val_ind = list(map(int, l2[1:-1].split(",")))

In [11]:
from src.datasets.BACH import BACH
from torch_geometric.loader.dataloader import DataLoader

train_set, val_set = BACH(src_folder, ids=train_ind), BACH(src_folder, ids=val_ind)

def get_class(path_name):
    if "Benign" in path_name:
        return 0
    if "InSitu" in path_name:
        return 1
    if "Invasive" in path_name:
        return 2
    return 3

In [12]:
from src.predict_cancer import predict_cancer

#... run predict_cancer on each image (all 400), and save to a csv along with ground truth.

# ... seperate predictions based on train or val. Create Confusion Matrix

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


## Confusion Matrix